# Comments

* I used Chris datasets (tf records 256, 384, 512 and 512 with external data) to make efficientnet models.The model is posted here https://www.kaggle.com/ragnar123/efficientnet-x-384. In this public notebook the cv is 0.919 and the lb is 0.937. I made another experiment with a different seed and it got lb 0.925. This suggest that my cv is not align with the public test set and i should not trust on that score, so making a good cv strategy is very important.

* Here im stacking 19 models predictions, and used groupkfold with patient id as the group (test set is formed by patients that are not in the train set so this sounds a good strategy)

* For experimentation purpose i will run this script 5 times changing the seed and check the cv and lb, i hope this is more stable than my publick cv.

Here are the results

CV1: 0.9439, LB1: 0.940, SEED: 42

CV2: 0.9423, LB2: 0.939, SEED: 100

CV3: 0.9456, LB3: 0.940, SEED: 999

CV4: 0.9446, LB4: 0.941, SEED: 6

Let's add the picture size.

CV5: Look the end of the script, LB5: Look at the end of the script, SEED: 22

This ensembles with different seed are more stable than our initial model (lb 0.937)

I hope this experiment helps you in finding the best cv strategy, make confident predictions and dont overfitt the public leadearboard

In [ ]:
import numpy as np 
import pandas as pd
from tqdm import tqdm
from PIL import Image
import random
import os
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
from sklearn.model_selection import GroupKFold, StratifiedKFold, KFold
from sklearn import metrics
from sklearn import preprocessing
from bayes_opt import BayesianOptimization
from catboost import CatBoostClassifier
import gc

In [ ]:
## function to seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
# define a seed
SEED = 22
    
# let´s start seeding everything
seed_everything(SEED)

# function to read data and image data models predictions
def read_data():
    train = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/train.csv')
    test = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/test.csv')
    sub = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/sample_submission.csv')
    train1 = pd.read_csv('../input/melanoma-subs/EfficientNetB0_256.csv')
    train2 = pd.read_csv('../input/melanoma-subs/EfficientNetB0_512EX.csv')
    train3 = pd.read_csv('../input/melanoma-subs/EfficientNetB1_256.csv')
    train4 = pd.read_csv('../input/melanoma-subs/EfficientNetB1_384.csv')
    train5 = pd.read_csv('../input/melanoma-subs/EfficientNetB1_512.csv')
    train6 = pd.read_csv('../input/melanoma-subs/EfficientNetB1_512EX.csv')
    train7 = pd.read_csv('../input/melanoma-subs/EfficientNetB2_256.csv')
    train8 = pd.read_csv('../input/melanoma-subs/EfficientNetB2_384.csv')
    train9 = pd.read_csv('../input/melanoma-subs/EfficientNetB2_512.csv')
    train10 = pd.read_csv('../input/melanoma-subs/EfficientNetB2_512EX.csv')
    train11 = pd.read_csv('../input/melanoma-subs/EfficientNetB3_256.csv')
    train12 = pd.read_csv('../input/melanoma-subs/EfficientNetB3_384.csv')
    train13 = pd.read_csv('../input/melanoma-subs/EfficientNetB3_512.csv')
    train14 = pd.read_csv('../input/melanoma-subs/EfficientNetB4_256.csv')
    train15 = pd.read_csv('../input/melanoma-subs/EfficientNetB4_384.csv')
    train16 = pd.read_csv('../input/melanoma-subs/EfficientNetB4_512.csv')
    train17 = pd.read_csv('../input/melanoma-subs/EfficientNetB5_256.csv')
    train18 = pd.read_csv('../input/melanoma-subs/EfficientNetB5_384.csv')
    train19 = pd.read_csv('../input/melanoma-subs/EfficientNetB6_384.csv')
    test1 = pd.read_csv('../input/melanoma-subs/sub_EfficientNetB0_256.csv')
    test2 = pd.read_csv('../input/melanoma-subs/sub_EfficientNetB0_512EX.csv')
    test3 = pd.read_csv('../input/melanoma-subs/sub_EfficientNetB1_256.csv')
    test4 = pd.read_csv('../input/melanoma-subs/sub_EfficientNetB1_384.csv')
    test5 = pd.read_csv('../input/melanoma-subs/sub_EfficientNetB1_512.csv')
    test6 = pd.read_csv('../input/melanoma-subs/sub_EfficientNetB1_512EX.csv')
    test7 = pd.read_csv('../input/melanoma-subs/sub_EfficientNetB2_256.csv')
    test8 = pd.read_csv('../input/melanoma-subs/sub_EfficientNetB2_384.csv')
    test9 = pd.read_csv('../input/melanoma-subs/sub_EfficientNetB2_512.csv')
    test10 = pd.read_csv('../input/melanoma-subs/sub_EfficientNetB2_512EX.csv')
    test11 = pd.read_csv('../input/melanoma-subs/sub_EfficientNetB3_256.csv')
    test12 = pd.read_csv('../input/melanoma-subs/sub_EfficientNetB3_384.csv')
    test13 = pd.read_csv('../input/melanoma-subs/sub_EfficientNetB3_512.csv')
    test14 = pd.read_csv('../input/melanoma-subs/sub_EfficientNetB4_256.csv')
    test15 = pd.read_csv('../input/melanoma-subs/sub_EfficientNetB4_384.csv')
    test16 = pd.read_csv('../input/melanoma-subs/sub_EfficientNetB4_512.csv')
    test17 = pd.read_csv('../input/melanoma-subs/sub_EfficientNetB5_256.csv')
    test18 = pd.read_csv('../input/melanoma-subs/sub_EfficientNetB5_384.csv')
    test19 = pd.read_csv('../input/melanoma-subs/sub_EfficientNetB6_384.csv')

    
    def print_roc_auc(df, model):
        roc_auc = metrics.roc_auc_score(df['target'], df['predictions'])
        print(f'Our model {model} out of folds roc auc score is {roc_auc}')
        print('-'*50)
        print('\n')
        
    print_roc_auc(train1, 1)
    print_roc_auc(train2, 2)
    print_roc_auc(train3, 3)
    print_roc_auc(train4, 4)
    print_roc_auc(train5, 5)
    print_roc_auc(train6, 6)
    print_roc_auc(train7, 7)
    print_roc_auc(train8, 8)
    print_roc_auc(train9, 9)
    print_roc_auc(train10, 10)
    print_roc_auc(train11, 11)
    print_roc_auc(train12, 12)
    print_roc_auc(train13, 13)
    print_roc_auc(train14, 14)
    print_roc_auc(train15, 15)
    print_roc_auc(train16, 16)
    print_roc_auc(train17, 17)
    print_roc_auc(train18, 18)
    print_roc_auc(train19, 19)
    
    def fix_predictions(train, test, model):
        test.columns = ['image_name', 'predictions_{}'.format(model)]
        train = train[['image_name', 'predictions']]
        train.columns = ['image_name', 'predictions_{}'.format(model)]
        return train, test
    
    train1, test1 = fix_predictions(train1, test1, 1)
    train2, test2 = fix_predictions(train2, test2, 2)
    train3, test3 = fix_predictions(train3, test3, 3)
    train4, test4 = fix_predictions(train4, test4, 4)
    train5, test5 = fix_predictions(train5, test5, 5)
    train6, test6 = fix_predictions(train6, test6, 6)
    train7, test7 = fix_predictions(train7, test7, 7)
    train8, test8 = fix_predictions(train8, test8, 8)
    train9, test9 = fix_predictions(train9, test9, 9)
    train10, test10 = fix_predictions(train10, test10, 10)
    train11, test11 = fix_predictions(train11, test11, 11)
    train12, test12 = fix_predictions(train12, test12, 12)
    train13, test13 = fix_predictions(train13, test13, 13)
    train14, test14 = fix_predictions(train14, test14, 14)
    train15, test15 = fix_predictions(train15, test15, 15)
    train16, test16 = fix_predictions(train16, test16, 16)
    train17, test17 = fix_predictions(train17, test17, 17)
    train18, test18 = fix_predictions(train18, test18, 18)
    train19, test19 = fix_predictions(train19, test19, 19)

    train = train.merge(train1, on = 'image_name').merge(train2, on = 'image_name').merge(train3, on = 'image_name').merge(train4, on = 'image_name').merge(train5, on = 'image_name').merge(train6, on = 'image_name').merge(train7, on = 'image_name').merge(train8, on = 'image_name').merge(train9, on = 'image_name').merge(train10, on = 'image_name').merge(train11, on = 'image_name').merge(train12, on = 'image_name').merge(train13, on = 'image_name').merge(train14, on = 'image_name').merge(train15, on = 'image_name').merge(train16, on = 'image_name').merge(train17, on = 'image_name').merge(train18, on = 'image_name').merge(train19, on = 'image_name')
    test = test.merge(test1, on = 'image_name').merge(test2, on = 'image_name').merge(test3, on = 'image_name').merge(test4, on = 'image_name').merge(test5, on = 'image_name').merge(test6, on = 'image_name').merge(test7, on = 'image_name').merge(test8, on = 'image_name').merge(test9, on = 'image_name').merge(test10, on = 'image_name').merge(test11, on = 'image_name').merge(test12, on = 'image_name').merge(test13, on = 'image_name').merge(test14, on = 'image_name').merge(test15, on = 'image_name').merge(test16, on = 'image_name').merge(test17, on = 'image_name').merge(test18, on = 'image_name').merge(test19, on = 'image_name')
    return train, test, sub

def feature_engineering(train, test):
    # size of images
    trn_images = train['image_name'].values
    trn_sizes = np.zeros((trn_images.shape[0], 2))
    for i, img_path in enumerate(tqdm(trn_images)):
        img = Image.open(os.path.join('/kaggle/input/siim-isic-melanoma-classification/jpeg/train/', f'{img_path}.jpg'))
        trn_sizes[i] = np.array([img.size[0], img.size[1]])
    test_images = test['image_name'].values
    test_sizes = np.zeros((test_images.shape[0],2))
    for i, img_path in enumerate(tqdm(test_images)):
        img = Image.open(os.path.join('/kaggle/input/siim-isic-melanoma-classification/jpeg/test/', f'{img_path}.jpg'))
        test_sizes[i] = np.array([img.size[0],img.size[1]])
    train['w'] = trn_sizes[:,0]
    train['h'] = trn_sizes[:,1]
    test['w'] = test_sizes[:,0]
    test['h'] = test_sizes[:,1]
    
    return train, test

def encode_categorical(train, test):
    for col in ['sex', 'anatom_site_general_challenge']:
        encoder = preprocessing.LabelEncoder()
        train[col].fillna('unknown', inplace = True)
        test[col].fillna('unknown', inplace = True)
        train[col] = encoder.fit_transform(train[col])
        test[col] = encoder.transform(test[col])
      # dont impute age, let light gradient boosting handle this
#     age_approx = np.nanmean(np.concatenate([np.array(train['age_approx']), np.array(test['age_approx'])]))
#     train['age_approx'].fillna(age_approx, inplace = True)
#     test['age_approx'].fillna(age_approx, inplace = True)
    train['patient_id'].fillna('unknown', inplace = True)
    return train, test

def train_and_evaluate_lgbm(train, test, params, verbose_eval, folds = 5):
    
    # define usefull features
    features = [col for col in train.columns if col not in ['image_name', 'patient_id', 'diagnosis', 'benign_malignant', 'target', 'source']]
    if verbose_eval != False:
        print('Training with features: ', features)
    
    
    # groupkfolds to predict evaluate unknown clients (just like the test set)
    kf = GroupKFold(n_splits = folds)
    target = 'target'
    
    oof_pred = np.zeros(len(train))
    y_pred = np.zeros(len(test))
     
    for fold, (tr_ind, val_ind) in enumerate(kf.split(train, groups = train['patient_id'])):
        if verbose_eval != False:
            print('\n')
            print('-'*50)
            print(f'Training fold {fold + 1}"')
        x_train, x_val = train[features].iloc[tr_ind], train[features].iloc[val_ind]
        y_train, y_val = train[target][tr_ind], train[target][val_ind]
        train_set = lgb.Dataset(x_train, y_train)
        val_set = lgb.Dataset(x_val, y_val)
        
        model = lgb.train(params, train_set, num_boost_round = 10000, early_stopping_rounds = 50, 
                         valid_sets = [train_set, val_set], verbose_eval = verbose_eval)
        
        
        oof_pred[val_ind] = model.predict(x_val)
        
        y_pred += model.predict(test[features]) / kf.n_splits
        
    rauc = metrics.roc_auc_score(train['target'], oof_pred)
    if verbose_eval != False:
        print(f'Our oof roc auc score for our lgbm model is {rauc}')
        
    gc.collect()
    
    return rauc, y_pred

train, test, sub = read_data()
train, test = feature_engineering(train, test)
train, test = encode_categorical(train, test)

In [ ]:
def run_lgb_bayesian(num_leaves, learning_rate, max_depth, lambda_l1, lambda_l2, bagging_fraction, bagging_freq, colsample_bytree, colsample_bynode, min_data_per_leaf, min_sum_hessian_per_leaf):
    
    params = {
        'boosting_type': 'gbdt',
        'metric': 'auc',
        'objective': 'binary',
        'n_jobs': -1,
        'seed': SEED,
        'num_leaves': int(num_leaves),
        'learning_rate': learning_rate,
        'max_depth': int(max_depth),
        'lambda_l1': lambda_l1,
        'lambda_l2': lambda_l2,
        'bagging_fraction': bagging_fraction,
        'bagging_freq': int(bagging_freq),
        'colsample_bytree': colsample_bytree,
        'colsample_bynode': colsample_bynode,
        'min_data_per_leaf': int(min_data_per_leaf),
        'min_sum_hessian_per_leaf': min_sum_hessian_per_leaf,
        'verbose': 0
    }
    
    rauc, y_pred = train_and_evaluate_lgbm(train, test, params, False)
    return rauc


# run bayesian optimization with optimal features
bounds_lgb = {
    'num_leaves': (20, 500),
    'learning_rate': (0.01, 0.2),
    'max_depth': (8, 250),
    'lambda_l1': (0, 3),
    'lambda_l2': (0, 3),
    'bagging_fraction': (0.4, 1),
    'bagging_freq': (1, 10),
    'colsample_bytree': (0.4, 1),
    'colsample_bynode': (0.4, 1),
    'min_data_per_leaf': (10, 100),
    'min_sum_hessian_per_leaf': (0.0001, 0.01)
}

lgb_bo = BayesianOptimization(run_lgb_bayesian, bounds_lgb, random_state = SEED)
lgb_bo.maximize(init_points = 300, n_iter = 300, acq = 'ucb', xi = 0.0, alpha = 1e-6)

params = {
    'boosting_type': 'gbdt',
    'metric': 'auc',
    'objective': 'binary',
    'n_jobs': -1,
    'seed': SEED,
    'num_leaves': int(lgb_bo.max['params']['num_leaves']),
    'learning_rate': lgb_bo.max['params']['learning_rate'],
    'max_depth': int(lgb_bo.max['params']['max_depth']),
    'lambda_l1': lgb_bo.max['params']['lambda_l1'],
    'lambda_l2': lgb_bo.max['params']['lambda_l2'],
    'bagging_fraction': lgb_bo.max['params']['bagging_fraction'],
    'bagging_freq': int(lgb_bo.max['params']['bagging_freq']),
    'colsample_bytree': lgb_bo.max['params']['colsample_bytree'],
    'colsample_bynode': lgb_bo.max['params']['colsample_bynode'],
    'min_data_per_leaf': int(lgb_bo.max['params']['min_data_per_leaf']),
    'min_sum_hessian_per_leaf': lgb_bo.max['params']['min_sum_hessian_per_leaf']
}


# train with new hyperparameters
roc_auc, y_pred = train_and_evaluate_lgbm(train, test, params, 50)

# predict
test['target'] = y_pred
sub = test[['image_name', 'target']]
sub.to_csv('lgbm_baseline_sub.csv', index = False)